In [1]:
import argparse
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import random
import numpy as np
import torch.nn as nn
import torch
from transformers import AutoModel
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score, classification_report
from transformers import AdamW, get_linear_schedule_with_warmup
# Any results you write to the current directory are saved as output.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from transformers import BertTokenizer,BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pack_padded_sequence
from torch.optim import AdamW
from tqdm import tqdm
from argparse import ArgumentParser
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.engine.engine import Engine, State, Events
from ignite.handlers import EarlyStopping
from ignite.contrib.handlers import TensorboardLogger, ProgressBar
from ignite.utils import convert_tensor
from torch.optim.lr_scheduler import ExponentialLR
import warnings  
warnings.filterwarnings('ignore')
import gc
import copy
import time
import random
import string

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold
from transformers import AutoTokenizer, AutoModel, AdamW
import random
import os
from urllib import request
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Helper Function

In [2]:
# Define a dataset class for prompted BERT fine tuning
# The dataset takes in query and passage, and then construct a training sample as:  <prompt> + [MASK] + <text>, returning the position of the mask
# It also performs padding and stores the labels
class BERTPromptDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, labels, tokenizer, prompt="ما هو شعور الكاتب؟", max_length=512):
        self.text=dataset['text']
        self.labels=labels
        # Construct the input sentence
#         input_sentences = ["{} {} {}".format(prompt, tokenizer.mask_token, text) for _, (text) in dataset.iterrows()]
        
#         # Encode and store
#         encodings_dict = tokenizer.batch_encode_plus(input_sentences, truncation=True, max_length=max_length, padding="max_length")
#         self.input_ids = encodings_dict['input_ids']
#         self.attn_masks = encodings_dict['attention_mask']
#         self.labels = labels

#         # Calculate the position of the mask using self.input_ids
#         mask_id = tokenizer.encode(tokenizer.mask_token)[1] # 103
#         self.mask_pos = [sent_ids.index(mask_id) for sent_ids in self.input_ids]
        
        
    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        return_dict = {"text": self.text[idx],                       
                       "labels": torch.tensor(self.labels[idx]).float()} 
        return return_dict

In [3]:
# # Class that expects a prompted input from the BERTPromptDataset
# # Takes the input, forward propagates it through BERT and concatenates the output at the [MASK] and [CLS] token to get a representation of the text
# # This is then passed to a linear head to perform binary classification for how relevant it is
# class BERTPrompt(torch.nn.Module):
#     def __init__(self, bert, tokenizer):
#         super().__init__()
#         self.bert = bert.cuda()
#         self.tokenizer = tokenizer
#         self.linear = torch.nn.Linear(768*2, 3).cuda()
#         self.act = torch.nn.Softmax()
        
#     # input_dict is obtained through indexing the dataset e.g. dataset[0:10]
#     def forward(self, input_ids,attention_mask,mask_pos ):
#         output = self.bert(input_ids,attention_mask)[0] # output is of shape [10, 256, 768]
#         cls_out = output[:, 0, :]
#         mask_out = output[torch.arange(cls_out.shape[0]), mask_pos, :] # indexing like [[0, 1], [13, 14]] will select items [[0, 13], [1, 14]]
        
#         representation = torch.cat([cls_out, mask_out], dim=1)
#         logit = self.linear(representation)
#         return logit
    
# #     # return the logit with sigmoid activation applied
# #     def predict(self, input_dict):
# #         return self.forward(input_dict)

In [4]:
# Class Plan:
# take in the length of the prompt
# create an embedding layer for that many tokens
# create an LSTM and MLP to process the prompt 
# How to feed the embedding to BERT
    # Get the embedding layer using https://huggingface.co/transformers/v3.0.2/model_doc/bert.html#transformers.BertModel.get_input_embeddings 
    # Then directly pass embedding to BERT using self.bert(input_embeds=, attention_mask=)
# add prompting logic to the tokenizer function
    # it needs to first pass all tokens through LSTM and MLP
    # then add them into the sentence as needed
class BERTPrompt(torch.nn.Module):
    def __init__(self, bert, tokenizer, max_length=512, prompt_length=5):
        super().__init__()
        self.hidden_size = 768
        self.max_length = max_length
        self.prompt_length = prompt_length
        self.tokenizer=tokenizer
        
        self.bert = bert.cuda()
        self.bert_embedding = self.bert.get_input_embeddings()
        self.linear = torch.nn.Linear(self.hidden_size, 3).cuda()
        self.act = torch.nn.Sigmoid() 
        
        # p tuning modules
        self.prompt_embedding = torch.nn.Embedding(prompt_length, self.hidden_size).cuda()
        self.lstm_head = torch.nn.LSTM(input_size=self.hidden_size, hidden_size=int(self.hidden_size/2),
            num_layers=2, bidirectional=True, batch_first=True).cuda() # takes (batch_size, sequence length, hidden_size)
        self.mlp_head = torch.nn.Sequential(torch.nn.Linear(self.hidden_size, self.hidden_size),
                                            torch.nn.ReLU(),
                                            torch.nn.Linear(self.hidden_size, self.hidden_size)).cuda()
        
    # calls the tokenize function to get input embeddings, then passes them through bert
    def forward(self, input_sents):
        embeds = torch.zeros(len(input_sents), self.max_length, self.hidden_size).cuda()
        att_mask = torch.zeros(len(input_sents), self.max_length).cuda().long()
        for i, sent in enumerate(input_sents):
            embeds[i, :, :], att_mask[i, :] = self.tokenize(sent)
        
        output = self.bert(inputs_embeds=embeds, attention_mask=att_mask)[0]
        cls_out = output[:, 0, :]
        logits = self.linear(cls_out)
        return logits
    
    # a tokenize function that embeds the sentence adds the prompt to the end and returns token embeddings along with attention mask
    def tokenize(self, input_sent,prompt='ما هو شعور الكاتب؟'):
        # generate prompt tokens from embedding
        self.prompt_length=len(tokenizer.encode(prompt))
        prompt_tokens = self.prompt_embedding(torch.arange(self.prompt_length).cuda())
        prompt_tokens = torch.unsqueeze(prompt_tokens, 0) # add a batch dimension
        prompt_tokens, _ = self.lstm_head(prompt_tokens)
        prompt_tokens = self.mlp_head(prompt_tokens)[0]
        
        # Encode the input_sentence
        encoding_dict = self.tokenizer.encode_plus(input_sent, truncation=True, max_length=self.max_length, padding="max_length")        
#         input_ids = encoding_dict["input_ids"]
#         mask_id = tokenizer.encode(tokenizer.mask_token)[1] # 103
        sep_pos = self.max_length - 1
        
        
#         sep_pos = encoding_dict["input_ids"].index(102) # the location of the [SEP] token is at the end of the input
        token_embeds = self.bert_embedding(torch.tensor(encoding_dict["input_ids"]).cuda())
        
        # Add the prompt tokens to the end and modify the att_mask to include the prompt
        start_prompt_pos = min(sep_pos, self.max_length - self.prompt_length - 1) # if the sentence is truncutated we must further truncate it to fit in the prompt
        end_prompt_pos = start_prompt_pos + self.prompt_length + 1
        token_embeds[start_prompt_pos:end_prompt_pos, :] = torch.cat([prompt_tokens, token_embeds[sep_pos:sep_pos+1, :]], dim=0)
        att_mask = encoding_dict["attention_mask"]
        att_mask[start_prompt_pos:end_prompt_pos] = [1]*(self.prompt_length+1)
        att_mask = torch.tensor(att_mask).cuda()
        
        return token_embeds, att_mask

In [5]:
from typing import Optional, Sequence

import torch
from torch import Tensor
from torch import nn
from torch.nn import functional as F


class FocalLoss(nn.Module):
    """ Focal Loss, as described in https://arxiv.org/abs/1708.02002.
    It is essentially an enhancement to cross entropy loss and is
    useful for classification tasks when there is a large class imbalance.
    x is expected to contain raw, unnormalized scores for each class.
    y is expected to contain class labels.
    Shape:
        - x: (batch_size, C) or (batch_size, C, d1, d2, ..., dK), K > 0.
        - y: (batch_size,) or (batch_size, d1, d2, ..., dK), K > 0.
    """

    def __init__(self,
                 alpha: Optional[Tensor] = None,
                 gamma: float = 0.,
                 reduction: str = 'mean',
                 ignore_index: int = -100):
        """Constructor.
        Args:
            alpha (Tensor, optional): Weights for each class. Defaults to None.
            gamma (float, optional): A constant, as described in the paper.
                Defaults to 0.
            reduction (str, optional): 'mean', 'sum' or 'none'.
                Defaults to 'mean'.
            ignore_index (int, optional): class label to ignore.
                Defaults to -100.
        """
        if reduction not in ('mean', 'sum', 'none'):
            raise ValueError(
                'Reduction must be one of: "mean", "sum", "none".')

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

        self.nll_loss = nn.NLLLoss(
            weight=alpha, reduction='none', ignore_index=ignore_index)

    def __repr__(self):
        arg_keys = ['alpha', 'gamma', 'ignore_index', 'reduction']
        arg_vals = [self.__dict__[k] for k in arg_keys]
        arg_strs = [f'{k}={v}' for k, v in zip(arg_keys, arg_vals)]
        arg_str = ', '.join(arg_strs)
        return f'{type(self).__name__}({arg_str})'

    def forward(self, x: Tensor, y: Tensor) -> Tensor:
        if x.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = x.shape[1]
            x = x.permute(0, *range(2, x.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            y = y.view(-1)

        unignored_mask = y != self.ignore_index
        y = y[unignored_mask]
        if len(y) == 0:
            return torch.tensor(0.)
        x = x[unignored_mask]

        # compute weighted cross entropy term: -alpha * log(pt)
        # (alpha is already part of self.nll_loss)
        log_p = F.log_softmax(x, dim=-1)
        ce = self.nll_loss(log_p, y)

        # get true class column from each row
        all_rows = torch.arange(len(x))
        log_pt = log_p[all_rows, y]

        # compute focal term: (1 - pt)^gamma
        pt = log_pt.exp()
        focal_term = (1 - pt)**self.gamma

        # the full loss: -alpha * ((1 - pt)^gamma) * log(pt)
        loss = focal_term * ce

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()

        return loss


def focal_loss(alpha: Optional[Sequence] = None,
               gamma: float = 0.,
               reduction: str = 'mean',
               ignore_index: int = -100,
               device='cuda',
               dtype=torch.float32) -> FocalLoss:
    """Factory function for FocalLoss.
    Args:
        alpha (Sequence, optional): Weights for each class. Will be converted
            to a Tensor if not None. Defaults to None.
        gamma (float, optional): A constant, as described in the paper.
            Defaults to 0.
        reduction (str, optional): 'mean', 'sum' or 'none'.
            Defaults to 'mean'.
        ignore_index (int, optional): class label to ignore.
            Defaults to -100.
        device (str, optional): Device to move alpha to. Defaults to 'cpu'.
        dtype (torch.dtype, optional): dtype to cast alpha to.
            Defaults to torch.float32.
    Returns:
        A FocalLoss object
    """
    if alpha is not None:
        if not isinstance(alpha, Tensor):
            alpha = torch.tensor(alpha)
        alpha = alpha.to(device=device, dtype=dtype)

    fl = FocalLoss(
        alpha=alpha,
        gamma=gamma,
        reduction=reduction,
        ignore_index=ignore_index)
    return fl

In [6]:
class F1_Loss(nn.Module):
    '''Calculate F1 score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. epsilon <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    - http://www.ryanzhang.info/python/writing-your-own-loss-function-module-for-pytorch/
    '''
    def __init__(self, epsilon=1e-7):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true,):
        # assert y_pred.ndim == 2
        # assert y_true.ndim == 1
        # print(y_pred.shape)
        # print(y_true.shape)
        # y_pred[y_pred<0.5]=0
        # y_pred[y_pred>=0.5]=0


        
        y_true_one_hot = F.one_hot(y_true.to(torch.int64), 3).to(torch.float32)
        # y_pred_one_hot = F.one_hot(y_pred.to(torch.int64), 2).to(torch.float32)
        
        tp = (y_true_one_hot * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true_one_hot) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true_one_hot) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true_one_hot * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        f1=f1.detach()
        # print(f1.shape)
        # y_pred=y_pred.reshape((y_pred.shape[0], 1))
        # y_true=y_true.reshape((y_true.shape[0], 1))

        # p1=y_true*(math.log(sigmoid(y_pred)))*(1-f1)[1]
        # p0=(1-y_true)*math.log(1-sigmoid(y_pred))*(1-f1)[0]


        # y_true_one_hot = F.one_hot(y_true.to(torch.int64), 2)
        # print(y_pred)
        # print(y_true_one_hot)
        CE =torch.nn.CrossEntropyLoss(weight=( 1 - f1))(y_pred, y_true)
        # loss = ( 1 - f1)  * CE
        return  CE.mean()

In [7]:
def criterion(outputs1,  targets):

    criterion = F1_Loss()
    loss = criterion(outputs1, targets)
    return loss

In [8]:
def run_training(model, optimizer, scheduler, device, num_epochs, train_loader, valid_loader):
    # To automatically log gradients
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = 0
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss,f1_score = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        
       
        
        # deep copy the model
        if f1_score >= best_epoch_loss:
            print(f"Validation Loss Improved ({best_epoch_loss} ---> {f1_score})")
            best_epoch_loss = f1_score
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/marbert_prompt/Loss_newwww.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print("Model Saved")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [9]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [10]:
def prepare_loaders(train,valid):
   
    
    train_dataset = BERTPromptDataset(train[['text']], train['label'], tokenizer)
    valid_dataset = BERTPromptDataset(valid[['text']], valid['label'], tokenizer)
    
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

    
    
    

In [11]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    prediction=[]
    true_prediction=[]
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        
        text_ids = data['text']
#         text_mask = data['text_mask'].to(device, dtype = torch.long)
#         mask_pos =data['mask_pos'].to(device, dtype = torch.long)
        targets = torch.tensor(data['labels']).to(device, dtype=torch.long)
        
        batch_size = len(text_ids)

        outputs = model(text_ids)
        prediction.append(F.softmax(outputs).to('cpu').numpy())
        true_prediction.append(targets.to('cpu').numpy())

        # outputs = outputs.argmax(dim=1)
        
        loss = criterion(outputs, targets)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr']) 
    
    gc.collect()
    prediction = np.concatenate(prediction)
    true_prediction = np.concatenate(true_prediction)
    prediction=np.argmax(np.array(prediction),axis=1)
    print(print_statistics(np.array(true_prediction),prediction))
    print(f1_score(np.array(true_prediction),prediction, average='macro'))

    
    return epoch_loss, f1_score(np.array(true_prediction),prediction, average='macro')

In [12]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        
        text_ids = data['text']
        targets = torch.tensor(data['labels']).to(device, dtype=torch.long)
        
        batch_size = len(text_ids)
        # print(targets)

        outputs = model(text_ids)
#         print(outputs.shape)
#         print(outputs)
        
        # print(outputs.shape)

        
        loss = criterion(outputs, targets)
        loss = loss / CONFIG['n_accumulate']
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

In [13]:
from sklearn.metrics import jaccard_score,f1_score,accuracy_score,recall_score,precision_score,classification_report
def print_statistics(y, y_pred):
    accuracy = accuracy_score(y, y_pred)
    precision =precision_score(y, y_pred, average='weighted')
    recall = recall_score(y, y_pred, average='weighted')
    f_score = f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(classification_report(y, y_pred))
    return accuracy, precision, recall, f_score

# Begin Training

In [14]:
train = pd.read_csv('/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/Dataset/NADI2022_Subtask2_TRAIN.tsv', sep='\t', lineterminator='\n')
valid = pd.read_csv('/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/Dataset/NADI2022_Subtask2_DEV.tsv', sep='\t', lineterminator='\n')

In [15]:
CONFIG = {"seed": 42,
          "epochs": 15,
          "train_batch_size": 8,
          "valid_batch_size": 64,
          "max_length": 512,
          "learning_rate": 4e-6,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-8,
          "T_max": 500,
          "weight_decay": 1e-8,
          "n_fold": 5,
          "n_accumulate": 1,
          "num_classes": 18,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          }

In [16]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['#3_label'] = le.fit_transform(train['#3_label'].values)
valid['#3_label'] = le.transform(valid['#3_label'].values)
train.rename(columns = {'#2_content':'text', '#3_label':'label'}, inplace = True)
valid.rename(columns = {'#2_content':'text', '#3_label':'label'}, inplace = True)


In [17]:
# train = pd.read_csv('/home/ahmed/Downloads/Imagenet_VLCQ/Nadi/Dataset/data_subtask2_aug.csv')


In [18]:
tokenizer= AutoTokenizer.from_pretrained('UBC-NLP/MARBERTv2')


In [19]:
arabert_model=AutoModel.from_pretrained('UBC-NLP/MARBERTv2')


Some weights of the model checkpoint at UBC-NLP/MARBERTv2 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
pBERT = BERTPrompt(arabert_model, tokenizer, prompt_length=25)
from transformers import BertModel,BertConfig,BertModel,BertTokenizerFast,get_cosine_schedule_with_warmup,BertForMaskedLM,get_cosine_with_hard_restarts_schedule_with_warmup,get_linear_schedule_with_warmup


In [22]:


    
# Create Dataloaders
train_loader, valid_loader = prepare_loaders(train,valid)

pBERT.to(CONFIG['device'])
torch.cuda.empty_cache()

# Define Optimizer and Scheduler
optimizer = AdamW(pBERT.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
scheduler = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=len(train_loader),num_training_steps=CONFIG['epochs']*len(train_loader))

model, history = run_training(pBERT, optimizer, scheduler,
                              device=CONFIG['device'],
                              num_epochs=CONFIG['epochs'],
                              train_loader=train_loader, valid_loader=valid_loader )


del model, history, train_loader, valid_loader
_ = gc.collect()
print()

[INFO] Using GPU: NVIDIA GeForce RTX 3080 Ti



100%|██████████████████████████████████████████████| 187/187 [00:37<00:00,  5.02it/s, Epoch=1, LR=4e-6, Train_Loss=1.02]
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|█████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.36it/s, Epoch=1, LR=4e-6, Valid_Loss=0.981]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.35it/s, Epoch=15, LR=0, Valid_Loss=1.02]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.23it/s, Epoch=15, LR=0, Valid_Loss=1.02]

Accuracy: 0.656
Precision: 0.648
Recall: 0.656
F_score: 0.650

              precision    recall  f1-score   support

           0       0.70      0.68      0.69       190
           1       0.49      0.41      0.45       113
           2       0.69      0.77      0.73       197

    accuracy                           0.66       500
   macro avg       0.63      0.62      0.62       500
weighted avg       0.65      0.66      0.65       500

(0.656, 0.6483627694253881, 0.656, 0.6502434508272145)
0.6217880102408092

Training complete in 0h 10m 23s
Best Loss: 0.6377



In [20]:


    
# Create Dataloaders
train_loader, valid_loader = prepare_loaders(train,valid)

pBERT.to(CONFIG['device'])
torch.cuda.empty_cache()

# Define Optimizer and Scheduler
optimizer = AdamW(pBERT.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
scheduler = fetch_scheduler(optimizer)

model, history = run_training(pBERT, optimizer, scheduler,
                              device=CONFIG['device'],
                              num_epochs=CONFIG['epochs'],
                              train_loader=train_loader, valid_loader=valid_loader )


del model, history, train_loader, valid_loader
_ = gc.collect()
print()

[INFO] Using GPU: NVIDIA GeForce RTX 3080 Ti



100%|███████████████████████████████████████████| 187/187 [00:42<00:00,  4.39it/s, Epoch=1, LR=1.39e-5, Train_Loss=0.93]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|                                                                                             | 0/8 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|██████████████████████████████████████████████| 8/8 [00:05<00:00,  1.58it/s, Epoch=1, LR=1.39e-5, Valid_Loss=0.791]


Accuracy: 0.668
Precision: 0.665
Recall: 0.668
F_score: 0.637

              precision    recall  f1-score   support

           0       0.66      0.82      0.73       190
           1       0.65      0.21      0.32       113
           2       0.68      0.79      0.73       197

    accuracy                           0.67       500
   macro avg       0.66      0.60      0.59       500
weighted avg       0.67      0.67      0.64       500

(0.668, 0.6652017027854406, 0.668, 0.6369118593320932)
0.5929438686627101
Validation Loss Improved (0 ---> 0.5929438686627101)
Model Saved



...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|████████████████████████████████████████| 187/187 [00:48<00:00,  3.85it/s, Epoch=15, LR=1.34e-5, Train_Loss=0.0135]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|                                                                                             | 0/8 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|██████████████████████████████████████████████| 8/8 [00:05<00:00,  1.58it/s, Epoch=15, LR=1.34e-5, Valid_Loss=1.65]


Accuracy: 0.658
Precision: 0.661
Recall: 0.658
F_score: 0.659

              precision    recall  f1-score   support

           0       0.71      0.70      0.70       190
           1       0.48      0.50      0.49       113
           2       0.72      0.71      0.71       197

    accuracy                           0.66       500
   macro avg       0.64      0.64      0.64       500
weighted avg       0.66      0.66      0.66       500

(0.658, 0.6608435461054668, 0.658, 0.6593104135966205)
0.6359678422896814

Training complete in 0h 13m 45s
Best Loss: 0.6484



In [20]:


    
# Create Dataloaders
train_loader, valid_loader = prepare_loaders(train,valid)

pBERT.to(CONFIG['device'])
torch.cuda.empty_cache()

# Define Optimizer and Scheduler
optimizer = AdamW(pBERT.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
scheduler = fetch_scheduler(optimizer)

model, history = run_training(pBERT, optimizer, scheduler,
                              device=CONFIG['device'],
                              num_epochs=CONFIG['epochs'],
                              train_loader=train_loader, valid_loader=valid_loader )


del model, history, train_loader, valid_loader
_ = gc.collect()
print()

[INFO] Using GPU: NVIDIA GeForce RTX 3080 Ti



100%|███████████████████████████████████████████| 187/187 [00:37<00:00,  5.03it/s, Epoch=1, LR=1.39e-5, Train_Loss=1.01]
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|██████████████████████████████████████████████| 8/8 [00:03<00:00,  2.17it/s, Epoch=1, LR=1.39e-5, Valid_Loss=0.924]


Accuracy: 0.614
Precision: 0.633
Recall: 0.614
F_score: 0.546

              precision    recall  f1-score   support

           0       0.74      0.66      0.70       190
           1       0.60      0.03      0.05       113
           2       0.55      0.90      0.68       197

    accuracy                           0.61       500
   macro avg       0.63      0.53      0.48       500
weighted avg       0.63      0.61      0.55       500

(0.614, 0.6330378280542986, 0.614, 0.5461965062666406)
0.4776132649306232
Validation Loss Improved (0 ---> 0.4776132649306232)
Model Saved



...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|███████████████████████████████████████████| 187/187 [00:37<00:00,  5.03it/s, Epoch=15, LR=1.34e-5, Train_Loss=0.6]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|█████████████████████████████████████████████| 8/8 [00:03<00:00,  2.23it/s, Epoch=15, LR=1.34e-5, Valid_Loss=0.931]

Accuracy: 0.612
Precision: 0.640
Recall: 0.612
F_score: 0.621

              precision    recall  f1-score   support

           0       0.69      0.63      0.66       190
           1       0.40      0.56      0.47       113
           2       0.73      0.62      0.67       197

    accuracy                           0.61       500
   macro avg       0.61      0.60      0.60       500
weighted avg       0.64      0.61      0.62       500

(0.612, 0.6397236123748862, 0.612, 0.6208748858447488)
0.5993911719939117

Training complete in 0h 10m 17s
Best Loss: 0.6384



In [20]:


    
# Create Dataloaders
train_loader, valid_loader = prepare_loaders(train,valid)

pBERT.to(CONFIG['device'])
torch.cuda.empty_cache()

# Define Optimizer and Scheduler
optimizer = AdamW(pBERT.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
scheduler = fetch_scheduler(optimizer)

model, history = run_training(pBERT, optimizer, scheduler,
                              device=CONFIG['device'],
                              num_epochs=CONFIG['epochs'],
                              train_loader=train_loader, valid_loader=valid_loader )


del model, history, train_loader, valid_loader
_ = gc.collect()
print()

[INFO] Using GPU: NVIDIA GeForce RTX 3080 Ti



100%|███████████████████████████████████████████| 187/187 [01:05<00:00,  2.86it/s, Epoch=1, LR=1.39e-5, Train_Loss=1.02]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|                                                                                             | 0/8 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|██████████████████████████████████████████████| 8/8 [00:06<00:00,  1.27it/s, Epoch=1, LR=1.39e-5, Valid_Loss=0.935]


Accuracy: 0.734
Precision: 0.791
Recall: 0.734
F_score: 0.685

              precision    recall  f1-score   support

           0       0.81      0.92      0.86        24
           1       1.00      0.14      0.25        14
           2       0.66      0.88      0.75        26

    accuracy                           0.73        64
   macro avg       0.82      0.65      0.62        64
weighted avg       0.79      0.73      0.68        64

(0.734375, 0.7912698412698413, 0.734375, 0.6845693707810993)
0.6222811528983178
Validation Loss Improved (0 ---> 0.6222811528983178)
Model Saved

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has alr

...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|█████████████████████████████████████████| 187/187 [01:00<00:00,  3.11it/s, Epoch=15, LR=1.34e-5, Train_Loss=0.594]
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|█████████████████████████████████████████████| 8/8 [00:05<00:00,  1.44it/s, Epoch=15, LR=1.34e-5, Valid_Loss=0.934]


Accuracy: 0.703
Precision: 0.696
Recall: 0.703
F_score: 0.690

              precision    recall  f1-score   support

           0       0.77      0.83      0.80        24
           1       0.62      0.36      0.45        14
           2       0.67      0.77      0.71        26

    accuracy                           0.70        64
   macro avg       0.69      0.65      0.66        64
weighted avg       0.70      0.70      0.69        64

(0.703125, 0.6960136217948718, 0.703125, 0.6896103896103896)
0.6562770562770562

Training complete in 0h 17m 33s
Best Loss: 0.7551

